In [3]:
% matplotlib inline
import mpld3
mpld3.enable_notebook()
from __future__ import print_function
import math
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact


def funct(x, y):
    '''
    f(x) component of equation y’=f(x) from 4th variant
    :param x: x-value
    :param y: y-value
    :return: result of function
    '''
    return 1 + (y * (2 * x + 1) / (math.pow(x, 2)))


def exact(X, x0, y0):
    '''
    computes analytical solution for each x in list X with given initial values (x0, y0) using analytical solution
    :param X: list of x-values from x0 to x with given step
    :param x0: initial x-value
    :param y0: initial y-value
    :return: result of function
    '''
    # list for analytical solutions
    YE = []
    # compute constant for initial value problem
    C = ((y0 + math.pow(x0, 2)) * math.exp(1 / x0)) / math.pow(x0, 2)
    for x in X:
        YE.append(((C * math.pow(x, 2)) / (math.exp(1 / x))) - math.pow(x, 2))
    return YE


def euler(X, y0, step):
    '''
    Euler numerical procedure for solving ordinary differential equations with a given initial value
    :param X: list of x-values from x0 to x
    :param y0: initial y-value on y-axis
    :param step: a grid step
    :return: result of euler method
    '''
    yn = y0
    # list with solutions
    Y = [y0]
    # computes solutions for each x in X
    for x in X:
        yn = yn + step * funct(x, yn)
        Y.append(yn)
    # exclude the last element of the list
    # because with x(n) computes y(n+1) solutions
    return Y[:len(Y) - 1]


def improved_euler(X, y0, step):
    '''
    Improved Euler numerical procedure for solving ordinary differential equations with a given initial value
    :param X: list of x-values from x0 to x with given step
    :param y0: initial y-value on y-axis
    :param step: a grid step
    :return: result of improved euler method
    '''
    yni = y0
    # list with solutions
    Y = [yni]
    # computes solutions for each x in X
    for x in X:
        k1 = funct(x, yni)
        k2 = funct(x + step, yni + (step * k1))
        yni = yni + ((step / 2) * (k1 + k2))
        Y.append(yni)
    # exclude the last element of the list
    # because with x(n) computes y(n+1) solutions
    return Y[:len(Y) - 1]


def runge_kutta(X, y0, step):
    '''
    Runge-Kutta's numerical procedure for solving ordinary differential equations with a given initial value
    :param X: list of x-values from x0 to x with given step
    :param y0: initial y-value on y-axis
    :param step: a grid step
    :return: result of Runge-Kutta method
    '''
    ynr = y0
    #list with solutions
    Y = [ynr]
    # computes solutions for each x in X
    for x in X:
        rk1 = funct(x, ynr)
        rk2 = funct(x + (step / 2), ynr + ((step / 2) * rk1))
        rk3 = funct(x + (step / 2), ynr + ((step / 2) * rk2))
        rk4 = funct(x + step, ynr + step * rk3)
        ynr = ynr + (step / 6) * (rk1 + 2 * rk2 + 2 * rk3 + rk4)
        Y.append(ynr)
    # exclude the last element of the list
    # because with x(n) computes y(n+1) solutions
    return Y[:len(Y) - 1]


def local_error(Y, YE):
    '''
    function that computes local truncation error of numerical method by subtracting approximated result from analytical one
    :param Y: list of solutions one of the numerical procedures
    :param YE: list of analytical solutions
    :return: list of local truncation errors produced by numerical method on range from x0 to x
    '''
    ER = []
    for i in range(len(Y)):
        ER.append(math.fabs(Y[i] - YE[i]))
    return ER


def global_error(x0, y0, x, P):
    '''
    function that computes the global truncation error by choosing the maximum local error
    on each step with partition from n0 to N
    :param x0: initial x-value
    :param y0: initial y-value
    :param x: final position on x-axis
    :param P: list of partitions from n0 to N with step 1
    :return: lists with truncation errors produced by euler, improved euler and Runge-Kutta methods respectively
    '''
    ERU = []
    ERI = []
    ERR = []
    for p in P:
        #computes step size for partition p
        h = (float)(x - x0) / p
        X = np.arange(x0, x + 0.001, h)
        YE = exact(X, x0, y0)
        Y = euler(X, y0, h)
        ERU.append(max(local_error(Y, YE)))
        YI = improved_euler(X, y0, h)
        ERI.append(max(local_error(YI, YE)))
        YR = runge_kutta(X, y0, h)
        ERR.append(max(local_error(YR, YE)))
    return ERU, ERI, ERR


def main(init_x, init_y, x, step, n0, N):
    '''
    computes solutions and plots the graphs
    :param init_x: initial x-value
    :param init_y: initial y-value
    :param x: final position on x-axis
    :param step: a grip step
    :param n0: initial partition
    :param N: final partition
    '''
    x0 = int(init_x)
    y0 = int(init_y)
    b = int(x)
    h = float(step)

    X = np.arange(x0, b + h, h)
    YE = exact(X, x0, y0)
    Y = euler(X, y0, h)
    YI = improved_euler(X, y0, h)
    YR = runge_kutta(X, y0, h)

    P = np.arange(int(n0), int(N) + 1, 1)
    ERU, ERI, ERR = global_error(x0, y0, b, P)

    fig = plt.figure(1, figsize=(10, 10))

    # plot figure of all 3 methods with analytical on one graph
    sub = fig.add_subplot(211)
    sub.set_title("Approximated solutions")
    sub.plot(X, YE, 'r', label="Exact")
    sub.plot(X, Y, 'g', label="Euler")
    sub.plot(X, YI, 'b', label="Improved")
    sub.plot(X, YR, 'y', label="Runge Kutta")
    sub.set_xlabel("x")
    sub.set_ylabel("y")
    sub.legend()

    # plot an error of each method on one graph
    sub = fig.add_subplot(212)
    sub.set_title("Global errors")
    sub.plot(P, ERU, 'g', label="Euler")
    sub.plot(P, ERI, 'b', label="Improved")
    sub.plot(P, ERR, 'y', label="Runge Kutta")
    sub.set_xlabel("N")
    sub.set_ylabel("y")
    sub.legend()

    plt.show()


# makes plots interactive
if __name__ == '__main__':
    interact(main, init_x='1', init_y='1', x='10', step='1', n0='9', N='90')

interactive(children=(Text(value='1', description='init_x'), Text(value='1', description='init_y'), Text(value…